In [2]:
import numpy as np
import time


index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [3]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    # ("DF", [0.1]),
    # ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [4]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA" and attack_name!= "AutoPGD" and attack_name!= "DF":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

unique_labels, counts = np.unique(y_label_0_1, return_counts=True)
print(unique_labels, counts)
valid_labels = unique_labels[counts > 1]
mask = np.isin(y_label_0_1, valid_labels)
x_test_adv_0_1 = x_test_adv_0_1[mask]
y_label_0_1 = y_label_0_1[mask]
print(np.unique(y_label_0_1, return_counts = True))


[ 1  2  3  4  5  6  8 10] [954961     14 238757      4     20 238756      1      5]
(array([ 1,  2,  3,  4,  5,  6, 10]), array([954961,     14, 238757,      4,     20, 238756,      5]))


In [5]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '10' encode 6


In [6]:
# Collect 20% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_20, _ , y_20 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.2, stratify=y_label_0_1, random_state=42)


In [7]:
X_20.shape

(286504, 42)

In [8]:
from xgboost import XGBClassifier

print('xgb')
xgb20 = XGBClassifier()
xgb20.fit(X_20, y_20)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf20 = RandomForestClassifier()
rf20.fit(X_20, y_20)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt20 = DecisionTreeClassifier()
# dt20.fit(X_20, y_20)

xgb
RF


RandomForestClassifier()

In [9]:
y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [ ]:
import time
import numpy as np




from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb20.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude3Attack/WUSTL_Input20/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude3Attack/WUSTL_Input20/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[2 6] [     7 238746]
(7, 42) (7,)
Save 2 to baseline_Def2.npy
(238746, 42) (238746,)
Save 6 to baseline_Def6.npy
Execution Time: 2.070382 seconds
(716259,)
[1 2 3 6] [478902      3  17013 220341]
(478902, 42) (478902,)
Save 1 to BIM_Def1.npy
(3, 42) (3,)
Save 2 to BIM_Def2.npy
(17013, 42) (17013,)
Save 3 to BIM_Def3.npy
(220341, 42) (220341,)
Save 6 to BIM_Def6.npy
Execution Time: 5.138366 seconds
(716259,)
[1 2 3 6] [495438     10    207 220604]
(495438, 42) (495438,)
Save 1 to FGSM_Def1.npy
(10, 42) (10,)
Save 2 to FGSM_Def2.npy
(207, 42) (207,)
Save 3 to FGSM_Def3.npy
(220604, 42) (220604,)
Save 6 to FGSM_Def6.npy
Execution Time: 3.135805 seconds
(716259,)
[1 2 3 6] [478902      3  17013 220341]
(478902, 42) (478902,)
Save 1 to PGD_Def1.npy
(3, 42) (3,)
Save 2 to PGD_Def2.npy
(17013, 42) (17013,)
Save 3 to PGD_Def3.npy
(220341, 42) (220341,)
Save 6 to PGD_Def6.npy
Execution Time: 2.673753 seconds
(716259,)
[1 2 3 6] [714283      9   1780    187]
(714283, 42) (714283,)
Sav

In [ ]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf20.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude3Attack/WUSTL_Input20/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude3Attack/WUSTL_Input20/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

In [ ]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt20.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_Exclude3Attack/WUSTL_Input20/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_Exclude3Attack/WUSTL_Input20/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [ ]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}20")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_Exclude3Attack/WUSTL_Input20/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [ ]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input20_Exclude3Attack.csv")

In [16]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB20,0,7,0,0,0,238746,0,0,0,0,0,238753
1,XGB20,478902,3,17013,0,0,220341,0,0,0,0,0,716259
2,XGB20,495438,10,207,0,0,220604,0,0,0,0,0,716259
3,XGB20,478902,3,17013,0,0,220341,0,0,0,0,0,716259
4,XGB20,714283,9,1780,0,0,187,0,0,0,0,0,716259
5,XGB20,423321,76,80162,5,11,212684,0,0,0,0,0,716259
6,XGB20,0,21,0,0,0,716238,0,0,0,0,0,716259
7,XGB20,204702,99770,30403,16,145,381223,0,0,0,0,0,716259
8,XGB20,28783,0,465737,0,0,221739,0,0,0,0,0,716259
9,XGB20,483796,2,12478,0,0,219983,0,0,0,0,0,716259
